<a href="https://colab.research.google.com/github/AbderrhmanAbdellatif/CRM/blob/master/Customer%20Lifetime%20Value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Calculating Lifetime Value is the easy part. First we need to select a time window. It can be anything like 3, 6, 12, 24 months. By the equation below, we can have Lifetime Value for each customer in that specific time window:

### Lifetime Value: Total Gross Revenue - Total Cost    >> ne demek istemis , Total Gross Revenue nedir ,   Total Cost nedir

### This equation now gives us the historical lifetime value. If we see some customers having very high negative lifetime value historically, it could be too late to take an action. At this point, we need to predict the future with machine learning:

In [0]:
#import libraries
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
#import machine learning related libraries
from sklearn.svm import SVC 
from sklearn.multioutput import MultiOutputClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
import xgboost as xgb 
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import plotly.offline as pyoff
import plotly.graph_objs as go
import xgboost as xgb
import datetime as dt
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import xgboost as xgb

#initate plotly
pyoff.init_notebook_mode()

In [0]:
#read data from csv and redo the data work we done before
tx_data = pd.read_csv('https://raw.githubusercontent.com/AbderrhmanAbdellatif/CRM/master/OnlineRetail.csv',encoding= 'unicode_escape')
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])
tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)
tx_uk.head()

In [0]:
filtered_data=tx_data[['Country','CustomerID']].drop_duplicates()
filtered_data.Country.value_counts()[:10].plot(kind='bar')

In [0]:
tx_data.describe()

In [0]:
uk_data = tx_data.describe()[(tx_data.describe()['Quantity']>0)]
uk_data=tx_data[['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]
uk_data['TotalPrice'] = uk_data['Quantity'] * uk_data['UnitPrice']
uk_data['InvoiceDate'].min(),uk_data['InvoiceDate'].max()

In [0]:
PRESENT = dt.datetime(2011,12,10)
uk_data['InvoiceDate'] = pd.to_datetime(uk_data['InvoiceDate'])
uk_data.head()

In [0]:
rfm= uk_data.groupby('CustomerID').agg({'InvoiceDate': lambda date: (PRESENT - date.max()).days,
                                        'InvoiceNo': lambda num: len(num),
                                        'TotalPrice': lambda price: price.sum()})

In [0]:
rfm.columns

In [0]:
rfm.columns=['recency','frequency','monetary']
rfm['recency'] = rfm['recency'].astype(int)
rfm

In [0]:
wcss = {}
for k in range(1,11):#30 adet orenklerememiz oldugu icin 1'den 30'a kadar gitmesini sagliyoruz
    kmeans = KMeans(n_clusters= k, init= 'k-means++', max_iter= 300)
    kmeans.fit(rfm)# standartlastirdigimi datamiza gore fit et, yani modelimizi olustur.
    wcss[k] = kmeans.inertia_#inertia ne demek ? her bir k degeri icin wcss degerini bul  
sns.pointplot(x = list(wcss.keys()), y = list(wcss.values()))
wcss


Müşterilerimizi segmentlere ayırdık ve kimin en iyisi olduğunu öğrendik. Şimdi yakından izlememiz gereken en önemli metriklerden birini ölçme zamanı: 
**Müşteri Yaşam Boyu Değeri**

Gelir elde etmek ve kârlı olmak için müşterilere (edinme maliyetleri, çevrimdışı reklamlar, promosyonlar, indirimler vb.) Yatırım yapıyoruz. Doğal olarak, bu eylemler bazı müşterileri yaşam boyu değer açısından süper değerli kılar, ancak her zaman karlılığı azaltan bazı müşteriler vardır. Bu davranış kalıplarını tanımlamamız, müşterileri segmentlere ayırmamız ve buna göre hareket etmemiz gerekiyor.


Yaşam Boyu Değerin hesaplanması kolay bir parçadır. İlk önce bir zaman penceresi seçmeliyiz. 3, 6, 12, 24 ay gibi bir şey olabilir. Aşağıdaki denklemle, belirli bir zaman penceresinde her müşteri için Yaşam Boyu Değere sahip olabiliriz:


**Ömür Boyu Değer**: Toplam Brüt Gelir - Toplam Maliyet

Bu denklem şimdi bize tarihsel yaşam boyu değer veriyor. Tarihsel olarak çok yüksek negatif yaşam boyu değere sahip bazı müşteriler görürsek, harekete geçmek için çok geç olabilir. Bu noktada, makine öğrenimi ile geleceği tahmin etmemiz gerekiyor 
**Müşterilerimizin yaşam boyu değerlerini tahmin eden basit bir makine öğrenimi modeli oluşturacağız.**

##Yaşam Boyu Değer Tahmini

*   Müşteri Yaşam Boyu Değer hesaplaması için uygun bir zaman dilimi tanımlayın
*   Geleceği tahmin etmek ve yaratmak için kullanacağımız özellikleri belirleyin
*   Makine öğrenimi modelini eğitmek için yaşam boyu değeri (YBD) hesapla
*   Makine öğrenimi modelini oluşturun ve çalıştırın
*   Modelin faydalı olup olmadığını kontrol edin


Zaman aralığına karar vermek sektörünüze, iş modelinize, stratejinize ve daha fazlasına bağlıdır. Bazı endüstriler için, 1 yıl çok uzun bir dönem iken, diğerleri için çok kısadır. Örneğimizde 6 ay ile devam edeceğiz 


Her bir müşteri kimliği için **RFM puanları** özellik kümesi için mükemmel adaylardır. Doğru bir şekilde uygulamak için veri kümemizi ayırmamız gerekir. 3 aylık veri alacağız, RFM'yi hesaplayacağız ve önümüzdeki 6 ayı tahmin etmek için kullanacağız. Bu yüzden önce iki veri çerçevesi oluşturmalı ve onlara RFM puanları eklemeliyiz.

# Create 3m and 6m dataframes

In [0]:
tx_3m = tx_uk[(tx_uk.InvoiceDate < datetime(2011,6,1)) & (tx_uk.InvoiceDate >= datetime(2011,3,1))].reset_index(drop=True)
tx_3m.head()

In [0]:
tx_6m = tx_uk[(tx_uk.InvoiceDate >= datetime(2011,6,1)) & (tx_uk.InvoiceDate < datetime(2011,12,1))].reset_index(drop=True)
tx_6m.head()

In [0]:
tx_user = pd.DataFrame(tx_3m['CustomerID'].unique())
tx_user.columns = ['CustomerID']
tx_user.head()

#Cluster Method

# Order cluster method

In [0]:
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

# calculate recency score

In [0]:
tx_max_purchase = tx_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
tx_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days
tx_user = pd.merge(tx_user, tx_max_purchase[['CustomerID','Recency']], on='CustomerID')

In [0]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] = kmeans.predict(tx_user[['Recency']])

tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)

In [0]:
tx_user.head()

In [0]:
plt.scatter(tx_user['Recency'],tx_user['RecencyCluster'],c=tx_user['CustomerID'],s=200,cmap='Blues')
plt.xlabel('Recency')
plt.ylabel('Cluster')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.show()

# calcuate frequency score

In [0]:
tx_frequency = tx_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
tx_frequency.columns = ['CustomerID','Frequency']
tx_user = pd.merge(tx_user, tx_frequency, on='CustomerID')

In [0]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster'] = kmeans.predict(tx_user[['Frequency']])
tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)

In [0]:
tx_user.head()

In [0]:
plt.scatter(tx_user['Frequency'],tx_user['FrequencyCluster'],c=tx_user['CustomerID'],s=200,cmap='Blues')
plt.xlabel('Frequency')
plt.ylabel('FrequencyCluster')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.show()

## calcuate revenue score


In [0]:
tx_3m['Revenue'] = tx_3m['UnitPrice'] * tx_3m['Quantity']
tx_revenue = tx_3m.groupby('CustomerID').Revenue.sum().reset_index()
tx_user = pd.merge(tx_user, tx_revenue, on='CustomerID')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Revenue']])
tx_user['RevenueCluster'] = kmeans.predict(tx_user[['Revenue']])
tx_user = order_cluster('RevenueCluster', 'Revenue',tx_user,True)

In [0]:
plt.scatter(tx_user['Revenue'],tx_user['RevenueCluster'],c=tx_user['CustomerID'],s=200,cmap='Blues')
plt.xlabel('Revenue')
plt.ylabel('RevenueCluster')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.show()

In [0]:
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['RevenueCluster']
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>2,'Segment'] = 'Mid-Value' 
tx_user.loc[tx_user['OverallScore']>4,'Segment'] = 'High-Value' 

In [0]:
tx_user

In [0]:
plt.scatter(tx_user['Segment'],tx_user['CustomerID'],c=tx_user['OverallScore'],s=300,cmap='Blues')
plt.xlabel('OverallScore')
plt.ylabel('CustomerID')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.show()

## calculate revenue and create a new dataframe for it

In [0]:
tx_6m['Revenue'] = tx_6m['UnitPrice'] * tx_6m['Quantity']
tx_user_6m = tx_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
tx_user_6m.columns = ['CustomerID','m6_Revenue']

In [0]:
plt.scatter(tx_user_6m['CustomerID'],tx_user_6m['m6_Revenue'],s=200,cmap='Blues')
plt.xlabel('CustomerID')
plt.ylabel('m6_Revenue')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.show()

![alt text](https://miro.medium.com/max/1400/1*3w0sJSAFCRQjhGKSG7ICYg.png)

### Histogram clearly shows we have customers with negative LTV. We have some outliers too. Filtering out the outliers makes sense to have a proper machine learning model.

# We will merge our 3 months and 6 months dataframes to see correlations between LTV and the feature set we have.

In [0]:
tx_merge = pd.merge(tx_user, tx_user_6m, on='CustomerID', how='left')

In [0]:
tx_merge.head()

In [0]:
tx_graph = tx_merge.query("m6_Revenue < 30000")

In [0]:
tx_graph.head()

özellik setimizi ve LTV verilerimizi birleştirir ve LTV ile genel RFM skorunu çizer

In [0]:
plt.scatter(tx_graph['OverallScore'],tx_graph['m6_Revenue'], c="g",s=200)
plt.xlabel('OverallScore')
plt.ylabel('m6_Revenue')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.legend(loc='upper left')
plt.show()

![alt text](https://miro.medium.com/max/1400/1*TTdNY2WV4n2yr_0IGD8bJA.png)

# The code below merges our feature set and LTV data and plots LTV vs overall RFM score:

Pozitif korelasyon burada oldukça görünür. Yüksek RFM puanı yüksek LTV anlamına gelir.

Makine öğrenimi modelini oluşturmadan önce, bu makine öğrenimi sorununun türünü belirlememiz gerekir. LTV'nin kendisi bir gerileme problemidir. Bir makine öğrenimi modeli LTV'nin $ değerini tahmin edebilir. Ancak burada LTV segmentleri istiyoruz. Çünkü diğer insanlarla iletişim kurmayı daha eyleme geçirilebilir ve kolay hale getirir. K-araçları kümelemesi uygulayarak, mevcut LTV gruplarımızı belirleyebilir ve bunun üzerine segmentler oluşturabiliriz.

In [0]:
#remove outliers
tx_merge = tx_merge[tx_merge['m6_Revenue']<tx_merge['m6_Revenue'].quantile(0.99)]

In [0]:
tx_merge.head()

In [0]:
wcss = {}
for k in range(1,11):#30 adet orenklerememiz oldugu icin 1'den 30'a kadar gitmesini sagliyoruz
    kmeans = KMeans(n_clusters= k, init= 'k-means++', max_iter= 300)
    kmeans.fit(tx_merge[['m6_Revenue']])# standartlastirdigimi datamiza gore fit et, yani modelimizi olustur.
    wcss[k] = kmeans.inertia_#inertia ne demek ? her bir k degeri icin wcss degerini bul  
sns.pointplot(x = list(wcss.keys()), y = list(wcss.values()))
wcss

**Before building the machine learning model, we need to identify what is the type of this machine learning problem. LTV itself is a regression problem. A machine learning model can predict the $ value of the LTV. But here, we want LTV segments. Because it makes it more actionable and easy to communicate with, other people. By applying K-means clustering,**

**we can identify our existing LTV groups and build segments on top of it.Considering business part of this analysis, we need to treat customers differently based on their predicted LTV. For this example, we will apply clustering and have 3 segments**

`(number of segments really depends on your business dynamics and goals):`


```
*   Low LTV
*   Mid LTV
*   High LTV

```



**We are going to apply K-means clustering to decide segments and observe their characteristics:**

In [0]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(tx_merge[['m6_Revenue']])
tx_merge['LTVCluster'] = kmeans.predict(tx_merge[['m6_Revenue']])

In [0]:
tx_merge.head()

In [0]:
#order cluster number based on LTV
tx_merge = order_cluster('LTVCluster', 'm6_Revenue',tx_merge,True)
tx_merge.head()

In [0]:
plt.scatter(tx_merge['m6_Revenue'],tx_merge['LTVCluster'],c=tx_merge['CustomerID'],s=200,cmap='Blues')
plt.xlabel('LTVCluster')
plt.ylabel('m6_Revenue')
F = plt.gcf()
Size = F.get_size_inches()
F.set_size_inches(Size[0]*2, Size[1]*2, forward=True) # Set forward to True to resize window along with plot in figure.
plt.legend(loc='upper left')
plt.show()

In [0]:
#creatinga new cluster dataframe
tx_cluster = tx_merge.copy()
#see details of the clusters
tx_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

### We have finished LTV clustering and here are the characteristics of each clusters


##Makine öğrenimi modelini eğitmeden önce birkaç adım daha var:


* Bazı özellik mühendisliği yapmak gerekiyor. Kategorik sütunları sayısal sütunlara dönüştürmeliyiz.

* Özelliklerin etiketimiz olan LTV kümeleriyle korelasyonunu kontrol edeceğiz.

*  Özellik setimizi ve etiketimizi (LTV) X ve y olarak böleriz. X'i y'yi tahmin etmek için kullanırız.

*  Eğitim ve Test veri kümesi oluşturur. Makine öğrenme modelinin oluşturulması için eğitim seti kullanılacaktır. Modelimizi gerçek performansını görmek için Test setine uygulayacağız.




İlk satırdan başlayalım. get_dummies () yöntemi, kategorik sütunları 0–1 gösterime dönüştürür. Örnekle tam olarak ne yaptığını görün:

In [0]:
#convert categorical columns to numerical
tx_class = pd.get_dummies(tx_cluster)

In [0]:
tx_merge.head()

Bu get_dummies () öncesindeki veri setimizdi. Segment olan bir kategorik sütunumuz var. Get_dummies () uygulandıktan sonra ne olur:

In [0]:
tx_class.head()

Segment sütunu gitti, ancak onu temsil eden yeni sayısal sütunlarımız var. 0 ve 1 ile 3 farklı sütuna dönüştürdük ve makine öğrenme modelimiz için kullanılabilir hale getirdik.

###calculate and show correlations


In [0]:
corr_matrix = tx_class.corr()

In [0]:
corr_matrix #gorsel yap 

In [0]:
corr_matrix['LTVCluster'].sort_values(ascending=False)

###create X and y, X will be feature set and y is the label - LTV

In [0]:
X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = tx_class['LTVCluster']

In [0]:
X.head()

In [0]:
y.head()

###split training and test sets


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=56)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [0]:
X_train.head()

In [0]:
X_test.head()

In [0]:
 y_train.head()

In [0]:
y_test.head()

Let’s start with the first line. get_dummies() method converts categorical columns to 0–1 notations. See what it exactly does with the example:

In [0]:
tx_cluster.head()

This was our dataset before get_dummies(). We have one categorical column which is Segment. What happens after applying get_dummies():

In [0]:
tx_class.head()

Segment column is gone but we have new numerical ones which represent it. We have converted it to 3 different columns with 0 and 1 and made it usable for our machine learning model.

##Ensemble Methods

Another way that we can improve the performance of algorithms on this problem is by using ensemble methods. In this section we will evaluate four diﬀerent ensemble machine learning algorithms, two boosting and two bagging methods:

In [0]:
# Compare Algorithms
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('DTree', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
	kfold = model_selection.KFold(n_splits=2)
	cv_results = model_selection.cross_val_score(model,X,y, cv=kfold, scoring='accuracy')
	results.append(cv_results)
	names.append(name)
	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
	print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

#Feature Scaling

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Gaussian Naive Bayes classifier

**Gaussian NB is based on the Naive Bayes theorem with the assumption of conditional independence between every pair of features given the label of the target class. The Graph for the likelihood of the feature vectors is Gaussian.**

![Gaussian Naive Bayes classifier](https://www.codespeedy.com/wp-content/uploads/2019/12/iris_gaussian.png)




In [0]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#Logistic Regression 

**Logistic Regression is one of the basic and powerful classifiers used in the machine learning model used for binary as well as multiclass classification problems. Let us apply logistic regression in the same way as we have applied the GaussianNB on the Iris dataset that we have and will be printing the accuracy score for this model as well.**


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
classifier = LogisticRegression(solver = 'lbfgs',multi_class='auto')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#Decision tree classifier 
**Decision Tree classifier is a widely used classification technique where several conditions are put on the dataset in a hierarchical manner until the data corresponding to the labels is purely separated. **



In [0]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#KNN (k-nearest neighbors) classifier 

 KNN classifier is a very simple technique for classification and it is based upon the Euclidean distance between two data points calculated by taking the distance between the feature vector.
In case of the same distance between a data point and data points belonging to two or more different classes then, the next lowest distance is calculated and it is assumed that the data point will belong to that class. The formula to calculate Euclidean distance between two data points is:

![](https://www.codespeedy.com/wp-content/uploads/2019/12/iris_euclidean.png)

In [0]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3,algorithm='ball_tree')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#Support Vector Machine
Support Vector Machine or SVM is a classifier that classifies the data points into the classes(Supervised Learning) and separates those classes using a hyperplane.


In [0]:
from sklearn.svm import SVC
classifier = SVC(C=50,kernel='rbf',gamma=1)     
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Random Forest Classifier 


In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.values.reshape(len(y_test),1)),1))
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#SVM Multiclassification Model

In [0]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X_train))
StandardScaler()
#print(scaler.mean_)
print(scaler.transform(X_train))

In [0]:
clf = LinearSVC(random_state=0, tol=1e-5,max_iter=1200000,dual=False)
svm_mul_class_model = clf.fit(X_train, y_train)
svm_mul_class_model


In [0]:
print('Accuracy of SVM classifier on training set: {:.2f}'
       .format(svm_mul_class_model.score(X_train, y_train)))

In [0]:
print('Accuracy of SVM classifier on test set: {:.2f}'
       .format(svm_mul_class_model.score(X_test[X_train.columns], y_test)))

In [0]:

# Transforming non numerical labels into numerical labels
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()
# Dimension of Train and Test set 

# encoding train labels 
encoder.fit(y_train)
y_train = encoder.transform(y_train)

# encoding test labels 
encoder.fit(y_test)
y_test = encoder.transform(y_test)
names_of_predictors = list(X_train.columns.values)

# Scaling the Train and Test feature set 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#Hyperparameter tuning using grid search and cross validation


In [0]:
#Libraries to Build Ensemble Model : Random Forest Classifier 
# Create the parameter grid based on the results of random search 
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]                    

#Training SVM model using radial kernel

In [0]:
# Performing CV to tune parameters for best SVM fit 
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC
svm_model = GridSearchCV(SVC(), params_grid, cv=5)
svm_model.fit(X_train_scaled, y_train)

In [0]:
print(svm_model)

#Confusion Matrix and Accuracy Score

In [0]:
# View the accuracy score
print('Best score for training data:', svm_model.best_score_,"\n") 

In [0]:
# View the best parameters for the model found using grid search
print('Best C:',svm_model.best_estimator_.C,"\n") 
print('Best Kernel:',svm_model.best_estimator_.kernel,"\n")
print('Best Gamma:',svm_model.best_estimator_.gamma,"\n")

In [0]:
final_model = svm_model.best_estimator_
Y_pred = final_model.predict(X_test_scaled)
Y_pred_label = list(encoder.inverse_transform(Y_pred))

In [0]:
# Making the Confusion Matrix
#print(pd.crosstab(Y_test_label, Y_pred_label, rownames=['Actual Activity'], colnames=['Predicted Activity']))
print(confusion_matrix(y_test,Y_pred_label))
print("\n")
print(classification_report(y_test,Y_pred_label))

print("Training set score for SVM: %f" % final_model.score(X_train_scaled , y_train))
print("Testing  set score for SVM: %f" % final_model.score(X_test_scaled  , y_test ))

svm_model.score

#XGBoost Multiclassification Model

In [0]:
ltv_xgb_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1,objective= 'multi:softprob',n_jobs=-1).fit(X_train, y_train)

In [0]:
ltv_xgb_model

In [0]:
print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))

In [0]:
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

##We can identify that by looking at classification report:

In [0]:
y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

First we need to check our benchmark. Biggest cluster we have is cluster 0 which is 76.5% of the total base. If we blindly say, every customer belongs to cluster 0, then our accuracy would be 76.5%.

84% vs 76.5% tell us that our machine learning model is a useful one but needs some improvement for sure. We should find out where the model is failing.

Precision and recall are acceptable for 0. As an example, for cluster 0 (Low LTV), if model tells us this customer belongs to cluster 0, 90 out of 100 will be correct (precision). And the model successfully identifies 93% of actual cluster 0 customers (recall). We really need to improve the model for other clusters. For example, we barely detect 56% of Mid LTV customers. Possible actions to improve those points:

    Adding more features and improve feature engineering
    Try different models other than XGBoost
    Apply hyper parameter tuning to current model
    Add more data to the model if possible

#Performance Comparison
In the previous sections we have used the **accuracy_score()** method to measure the accuracy of the different algorithms. Now, we will use the `ClassificationReport` class provided by the **Yellowbrick** library to give us a visual report of how our models perform.

#GaussianNB

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(gnb, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In the code above, first we import the ClassificationReport class provided by the yellowbrick.classifier module. Next, an object visualizer of the type ClassificationReport is created. Here the first argument is the GaussianNB object gnb that was created while implementing the Naive-Bayes algorithm in the ‘Naive-Bayes’ section. The second argument contains the labels ‘Won’ and ‘Loss’ from the ‘Opportunity Result’ column from the sales_data dataframe.

Next, we use the fit() method to train the visualizer object. This is followed by the score() method, which uses gnb object to carry out predictions as per the GaussianNB algorithm and then calculate the accuracy score of the predictions made by this algorithm. Finally, we use the poof() method to draw a plot of the different scores for the GaussianNB algorithm. Notice how the different scores are laid out against each of the labels ‘Won’ and ‘Loss’; this enables us to visualize the scores across the different target classes.



#LinearSVC

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(svm_model, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

In the code above, first we imported the ClassificationReport class provided by the yellowbrick.classifier module. Next, an object visualizer of the type ClassificationReport was created. Here the first argument is the LinearSVC object svc_model, that was created while implementing the LinearSVC algorithm in the ‘LinearSVC’ section. The second argument contains the labels ‘Won’ and ‘Loss’ from the ‘Opportunity Result’ column from the sales_data dataframe.

Next, we used the fit() method to train the ‘svc_model’ object. This is followed by the score() method which uses the svc_model object to carry out predictions according to the LinearSVC algorithm and then calculate the accuracy score of the predictions made by this algorithm. Finally, we used the poof() method to draw a plot of the different scores for the LinearSVC algorithm.

#Decision tree classifier



In [0]:
DT

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(dt, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

#svm Multclass

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(clf, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

#Random Forest Classifier

In [0]:
classifier

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(classifier, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

#Logistic Regression

In [0]:
logreg

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(logreg, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

#KNN (k-nearest neighbors) classifier

In [0]:
clfKN

In [0]:
from yellowbrick.classifier import ClassificationReport
# Instantiate the classification model and visualizer
visualizer = ClassificationReport(clfKN, classes=['Won','Loss'])
visualizer.fit(X_train,y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data